In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression

from sqlalchemy import create_engine

# Replace these variables with your PostgreSQL connection details
db_host = "localhost"
db_port = 5432
db_name = "postgres"
db_user = "postgres"
db_password = "postgres"

# Create an SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Query data from the database using SQLAlchemy engine
# Define the table names
customer_bureau_table = 'customer_bureau_details'
company_table = 'company_table'

# Query data from both tables using a JOIN operation
query = f"""
    SELECT
        cb.credit_score,
        cb.salary_amt,
        cb.total_txn_amt,
        cb.total_refund_amt,
        cb.dfd_count,
        cb.npa_count,
        cb.vas_count,
        cb.instacred_score,
        cb.score,
        c.company_score
    FROM {customer_bureau_table} cb
    JOIN {company_table} c ON cb.company_id = c.company_id
"""
df = pd.read_sql_query(query, engine)


# Extract features (independent variables) and target variable
X_train =df[['credit_score', 'salary_amt', 'total_txn_amt', 'total_refund_amt',
          'dfd_count', 'npa_count', 'vas_count', 'instacred_score', 'company_score']]
y_train = df['score']

print(y_train)

# Create and train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Print the learned weights and intercept
weights = model.coef_
intercept = model.intercept_

print(f"Weights: {weights}")
print(f"Intercept: {intercept}")


0    3198.81
1    2188.44
2    5291.76
3    5523.26
4    6954.07
5    2033.13
6    7683.49
7    4624.82
8    7895.03
9    7308.51
Name: score, dtype: float64
Weights: [ 1.00000050e+01  9.99946251e-04 -4.64087945e-08 -3.51841280e-07
 -1.00000722e+01  1.52684035e-04  8.41301856e-05 -6.86636737e-06
 -8.35186882e-05]
Intercept: 0.01265064380277181


In [5]:
X_test =df[['credit_score', 'salary_amt', 'total_txn_amt', 'total_refund_amt',
          'dfd_count', 'npa_count', 'vas_count', 'instacred_score', 'company_score']]
predicted_scores = model.predict(X_test)

In [6]:
predicted_scores

array([3198.81, 2188.44, 5291.76, 5523.26, 6954.07, 2033.13, 7683.49,
       4624.82, 7895.03, 7308.51])

In [9]:
table_name='customer_bureau_details'
df['predicted_scores'] = predicted_scores
df.to_sql(table_name, engine, index=False, if_exists='replace')

# Commit changes
engine.dispose()

storing the model

In [10]:
import joblib

# Save the trained model to a file
joblib.dump(model, 'linear_regr_model_customer.joblib')


['linear_regr_model_customer.joblib']

loading the model

In [11]:
import joblib

# Load the trained model from the file
model = joblib.load('linear_regr_model_customer.joblib')


In [12]:
# Assuming X_production is your new dataset in production
predicted_scores_production = model.predict(X_test)

In [13]:
print(predicted_scores)

[3198.81 2188.44 5291.76 5523.26 6954.07 2033.13 7683.49 4624.82 7895.03
 7308.51]


Incremental training of the model based on feedback

In [ ]:
# Update existing model with new data incrementally
existing_model = joblib.load('linear_regr_model_customer.joblib')

for new_data_batch in new_data_batches:
    X_train =df[['credit_score', 'salary_amt', 'total_txn_amt', 'total_refund_amt',
          'dfd_count', 'npa_count', 'vas_count', 'instacred_score'], 'company_score']
    y_train = df['score']
    existing_model.partial_fit(X_train, y_train)

#saving the model
joblib.dump(existing_model, 'updated_linear_reg_model_customer.joblib')